# LAB | Hyperparameter Tuning

**Load the data**

Finally step in order to maximize the performance on your Spaceship Titanic model.

The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

So far we've been training and evaluating models with default values for hyperparameters.

Today we will perform the same feature engineering as before, and then compare the best working models you got so far, but now fine tuning it's hyperparameters.

In [39]:
import pandas as pd
import numpy as np
import time

import scipy.stats as st

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler

from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, BaggingRegressor, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier

from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, root_mean_squared_error

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [40]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [41]:
#Preprocessing
spaceship = spaceship.drop(columns=["Name", "Cabin", "PassengerId"])
spaceship = spaceship.dropna()

spaceship["CryoSleep"] = spaceship["CryoSleep"].astype(int)
spaceship["VIP"] = spaceship["VIP"].astype(int)
spaceship["Transported"] = spaceship["Transported"].astype(int)

spaceship = pd.get_dummies(spaceship, columns=["HomePlanet", "Destination"], drop_first=True)

features = spaceship.drop("Transported", axis=1)
target = spaceship["Transported"]

scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

- Now let's use the best model we got so far in order to see how it can improve when we fine tune it's hyperparameters.

In [42]:
#Perform Train Test Split
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=0)

- Evaluate your model

In [43]:
#Bagging Classifier (with replacements)
bagging_clf = BaggingClassifier(
    estimator=DecisionTreeClassifier(max_depth=20, random_state=0),
    n_estimators=100,
    max_samples=1000,  
    bootstrap=True,        
    random_state=0)

bagging_clf.fit(X_train, y_train)
bagging_preds = bagging_clf.predict(X_test)
bagging_acc = (bagging_preds == y_test).mean()
print(f"Bagging Classifier Accuracy:, {bagging_acc: .4f}")

Bagging Classifier Accuracy:,  0.8014


**Grid/Random Search**

For this lab we will use Grid Search.

- Define hyperparameters to fine tune.

In [44]:
parameter_grid = {"max_depth": [10, 50],
                  "min_samples_split": [4, 16],
                  "max_leaf_nodes": [250, 100],
                  "max_features": ["sqrt", "log2"]} # In example we're going to test 2 * 2 * 2 * 2 = 16 combinations of hyperparameters

# We create an instance or our machine learning model
dt = DecisionTreeClassifier(random_state=123)

# We need to set this two variables to be able to compute a confidence interval
confidence_level = 0.95
folds = 10

- Run Grid Search

In [45]:
# Now we need to create an intance of the GridSearchCV class
gs = GridSearchCV(dt, param_grid=parameter_grid, cv=folds, verbose=10) # Here the "cv" allows you to define the number of folds to use.

start_time = time.time()
gs.fit(X_train, y_train)
end_time = time.time()

print("\n")
print(f"Time taken to find the best combination of hyperparameters among the given ones: {end_time - start_time: .4f} seconds")
print("\n")


print(f"The best combination of hyperparameters has been: {gs.best_params_}")
print(f"The R2 is: {gs.best_score_: .4f}")

results_gs_df = pd.DataFrame(gs.cv_results_).sort_values(by="mean_test_score", ascending=False)

#print(results_df.head())
gs_mean_score = results_gs_df.iloc[0,-3]
gs_sem = results_gs_df.iloc[0,-2] / np.sqrt(10)

gs_tc = st.t.ppf(1-((1-confidence_level)/2), df=folds-1)
gs_lower_bound = gs_mean_score - ( gs_tc * gs_sem )
gs_upper_bound = gs_mean_score + ( gs_tc * gs_sem )

print(f"The R2 confidence interval for the best combination of hyperparameters is: \
    ({gs_lower_bound: .4f}, {gs_mean_score: .4f}, {gs_upper_bound: .4f}) ")

#display(results_df)

# Let's store the best model
best_model = gs.best_estimator_

Fitting 10 folds for each of 16 candidates, totalling 160 fits
[CV 1/10; 1/16] START max_depth=10, max_features=sqrt, max_leaf_nodes=250, min_samples_split=4
[CV 1/10; 1/16] END max_depth=10, max_features=sqrt, max_leaf_nodes=250, min_samples_split=4;, score=0.749 total time=   0.0s
[CV 2/10; 1/16] START max_depth=10, max_features=sqrt, max_leaf_nodes=250, min_samples_split=4
[CV 2/10; 1/16] END max_depth=10, max_features=sqrt, max_leaf_nodes=250, min_samples_split=4;, score=0.771 total time=   0.0s
[CV 3/10; 1/16] START max_depth=10, max_features=sqrt, max_leaf_nodes=250, min_samples_split=4
[CV 3/10; 1/16] END max_depth=10, max_features=sqrt, max_leaf_nodes=250, min_samples_split=4;, score=0.774 total time=   0.0s
[CV 4/10; 1/16] START max_depth=10, max_features=sqrt, max_leaf_nodes=250, min_samples_split=4
[CV 4/10; 1/16] END max_depth=10, max_features=sqrt, max_leaf_nodes=250, min_samples_split=4;, score=0.753 total time=   0.0s
[CV 5/10; 1/16] START max_depth=10, max_features=sqrt

- Evaluate your model

In [46]:
# Now is time evaluate the model in the test set
y_test = best_model.predict(X_test)
y_test = best_model.predict(X_test)

y_pred_test= best_model.predict(X_test)

print("\n")
print(f"Test MAE: {mean_absolute_error(y_pred_test, y_test): .4f}")
print(f"Test MSE: {mean_squared_error(y_pred_test, y_test): .4f}")
print(f"Test RMSE: {root_mean_squared_error(y_pred_test, y_test): .4f}")
print(f"Test R2 score:  {best_model.score(X_test, y_test): .4f}")
print("\n")



Test MAE:  0.0000
Test MSE:  0.0000
Test RMSE:  0.0000
Test R2 score:   1.0000


